#### Task 10
Write a calibration program to calibrate your cellphone camera. For generating the calibration images you can display a checkerboard in you computer monitor. How do you evaluate the quality of your estimation?

In [1]:
import cv2
import numpy as np
import glob
import os

class CameraCalibrator:
    def __init__(self, checkerboard_dims=(9, 6), square_size=1.0):
        self.checkerboard_dims = checkerboard_dims
        self.square_size = square_size

        # Prepare object points: (0,0,0), (1,0,0), ... (8,5,0)
        objp = np.zeros((checkerboard_dims[0] * checkerboard_dims[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:checkerboard_dims[0], 0:checkerboard_dims[1]].T.reshape(-1, 2)
        objp *= square_size
        self.objp = objp

        self.objpoints = []  # 3D points in world
        self.imgpoints = []  # 2D points in image

    def add_image(self, image_path):
        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find checkerboard corners
        ret, corners = cv2.findChessboardCorners(gray, self.checkerboard_dims, None)

        if ret:
            print(f"Found corners in {image_path}")
            self.objpoints.append(self.objp)
            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1),
                                        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            self.imgpoints.append(corners2)
            return True
        else:
            print(f"Checkerboard not found in {image_path}")
            return False

    def calibrate(self, image_shape):
        """
        Calibrates the camera.
        Returns: ret, K, dist, rvecs, tvecs
        """
        ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
            self.objpoints, self.imgpoints, image_shape[::-1], None, None
        )
        return ret, K, dist, rvecs, tvecs

    def reprojection_error(self, K, dist, rvecs, tvecs):
        total_error = 0
        total_points = 0
        for i in range(len(self.objpoints)):
            imgpoints2, _ = cv2.projectPoints(self.objpoints[i], rvecs[i], tvecs[i], K, dist)
            error = cv2.norm(self.imgpoints[i], imgpoints2, cv2.NORM_L2)
            total_error += error**2
            total_points += len(imgpoints2)
        mean_error = np.sqrt(total_error / total_points)
        return mean_error


In [ ]:
image_folder = "data/checkboard/"
image_paths = glob.glob(os.path.join(image_folder, "*.jpeg"))

calibrator = CameraCalibrator(checkerboard_dims=(9, 6), square_size=1.0)

for img_path in image_paths:
    calibrator.add_image(img_path)

# Use shape of last loaded image
img = cv2.imread(image_paths[0])
ret, K, dist, rvecs, tvecs = calibrator.calibrate(img.shape[:2])
error = calibrator.reprojection_error(K, dist, rvecs, tvecs)

print("\n--- Calibration Results ---")
print("Camera matrix (K):\n", K)
print("Distortion coefficients:\n", dist.ravel())
print(f"Reprojection RMSE error: {error:.4f} pixels")

Found corners in data/checkboard/5.jpeg
Found corners in data/checkboard/8.jpeg
Found corners in data/checkboard/2.jpeg
Found corners in data/checkboard/1.jpeg
Found corners in data/checkboard/4.jpeg
Found corners in data/checkboard/6.jpeg
Found corners in data/checkboard/9.jpeg
Found corners in data/checkboard/7.jpeg
Found corners in data/checkboard/3.jpeg

--- Calibration Results ---
Camera matrix (K):
 [[1.11521097e+03 0.00000000e+00 7.95001061e+02]
 [0.00000000e+00 1.11563980e+03 6.04106162e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [ 0.0731748  -0.24600488  0.00144774 -0.00067746  0.2947214 ]
Reprojection RMSE error: 0.3349 pixels
